In [2]:
import sys

f = open('F:\DL\IMDB\\reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('F:\DL\IMDB\\labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)
        

        
        
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1

correct,total = (0,0)
for iter in range(iterations):
    
    # train on first 24,000（在前24000条电影评论上进行训练）
    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid   嵌入层+sigmoid激活函数
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax   线性层+softmax激活函数

        layer_2_delta = layer_2 - y # compare pred with truth（将预测结果和真值进行比较）
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop（反向传播）

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))

Iter:0 Progress:95.99% Training Accuracy:0.8309166666666666%%
Iter:1 Progress:95.99% Training Accuracy:0.8655%36486768077%
Test Accuracy:0.854


In [6]:
tokens[0]

{'',
 '\n',
 '.',
 'a',
 'about',
 'adults',
 'age',
 'all',
 'and',
 'as',
 'at',
 'believe',
 'bromwell',
 'burn',
 'can',
 'cartoon',
 'classic',
 'closer',
 'comedy',
 'down',
 'episode',
 'expect',
 'far',
 'fetched',
 'financially',
 'here',
 'high',
 'i',
 'immediately',
 'in',
 'insightful',
 'inspector',
 'is',
 'isn',
 'it',
 'knew',
 'lead',
 'life',
 'line',
 'm',
 'many',
 'me',
 'much',
 'my',
 'of',
 'one',
 'other',
 'pathetic',
 'pettiness',
 'pity',
 'pomp',
 'profession',
 'programs',
 'ran',
 'reality',
 'recalled',
 'remind',
 'repeatedly',
 'right',
 's',
 'sack',
 'same',
 'satire',
 'saw',
 'school',
 'schools',
 'scramble',
 'see',
 'situation',
 'some',
 'student',
 'students',
 'such',
 'survive',
 't',
 'teachers',
 'teaching',
 'than',
 'that',
 'the',
 'their',
 'think',
 'through',
 'time',
 'to',
 'tried',
 'welcome',
 'what',
 'when',
 'which',
 'who',
 'whole',
 'years',
 'your'}

In [7]:
len(tokens[0])

94

In [3]:
from collections import Counter
import math 

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)




In [4]:
print(similar('beautiful'))

[('beautiful', -0.0), ('liked', -0.7259811252367809), ('episodes', -0.7290309967452014), ('fantastic', -0.733280638859789), ('each', -0.7369376852405174), ('awesome', -0.744138755209802), ('appreciate', -0.7558663087586555), ('definitely', -0.7595374697012539), ('unexpected', -0.7616395323137153), ('simple', -0.7637285718379527)]


In [5]:
print(similar('terrible'))

[('terrible', -0.0), ('worse', -0.7570555094621823), ('mess', -0.7622124324661071), ('boring', -0.7908058323881745), ('laughable', -0.8058549930529031), ('avoid', -0.8077949514853672), ('disappointment', -0.8078155222511667), ('poor', -0.8109702158207468), ('dull', -0.8113694959717213), ('horrible', -0.8168321365960421)]
